In [2]:
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6 

from moSlicer import MoSlicer
import moMetrics as mm
import moSummaryMetrics as ms
import moPlots as moPlots
import moMetricBundle as mmb
import lsst.sims.maf.plots as plots

/Users/lynnej/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [3]:
# Define data set and generic metadata.
orbitfile = 'pha20141031.des'
#testDir = 'kraken_1039_pha20141031'
#testDir = 'kraken_1040_comb_pha20141031'
testDir = 'lucy_1000_pha20141031'
obsfile = os.path.join(testDir, 'pha20141031_allObs.txt')
runName = testDir.replace('_pha20141031', '')
metadata = 'MPC PHAs'

albedo = 0.13 
# the mean albedo is probably 0.07 but 0.13 is the preferred value for evaluating the size of unknown PHAs

In [4]:
# Read orbit and observation data into a moving object slicer.
mos = MoSlicer(orbitfile, Hrange=np.arange(15, 26, 0.5))
mos.readObs(obsfile)

In [5]:
summaryMetrics = [ms.CompletenessMetric(), ms.CumulativeCompletenessMetric()]

In [6]:
# Set up a metric bundle to identify discovery (and then calculate completeness)
# Calculate this for a variety of years.
discbundles = {}
extrapairbundles = {}
longdiscbundles = {}
magicbundles = {}
velocitybundles = {}
nyears = [5, 10, 12, 15]
highvelocity = 1.5  #deg/day
for yr in nyears:
    # 'normal' discovery conditions - 3 pairs in 15 nights.
    metric = mm.DiscoveryChancesMetric(tNight=90./60./24., nObsPerNight=2, 
                                        nNightsPerWindow=3, tWindow=15)
    slicer = mos
    constraint = 'night < %d' %(yr * 365 + 1)
    md = metadata + ' year %d, 3 pairs in 15 nights' %(yr)
    plotDict = {'nxbins':200, 'nybins':200, 'albedo':albedo}
    discbundles[yr] = mmb.MoMetricBundle(metric, slicer, constraint, 
                                        runName=runName, metadata=md, 
                                        plotDict=plotDict, summaryMetrics=summaryMetrics)
    # 'extra pair' discovery conditions - 4 pairs in 20 nights.
    metric = mm.DiscoveryChancesMetric(tNight=90./60./24., nObsPerNight=2, 
                                      nNightsPerWindow=4, tWindow=20)
    md = metadata + ' year %d, 4 pairs in 20 nights' %(yr)
    extrapairbundles[yr] = mmb.MoMetricBundle(metric, slicer, constraint, 
                                             runName=runName, metadata=md, 
                                             plotDict=plotDict, summaryMetrics=summaryMetrics)
    # 'long' discovery window - 3 pairs in 30 nights.
    metric = mm.DiscoveryChancesMetric(tNight=90./60./24., nObsPerNight=2,
                                      nNightsPerWindow=3, tWindow=30)
    md = metadata + ' year %d, 3 pairs in 30 nights' %(yr)
    longdiscbundles[yr] = mmb.MoMetricBundle(metric, slicer, constraint, 
                                             runName=runName, metadata=md, 
                                             plotDict=plotDict, summaryMetrics=summaryMetrics)
    # 'magic' discoveries - 6 visits within 30 nights
    metric = mm.MagicDiscoveryMetric(nObs=6, tWindow=60)
    md = metadata + ' year %d, 6 obs in 60 nights' %(yr)
    magicbundles[yr] = mmb.MoMetricBundle(metric, slicer, constraint,
                                         runName=runName, metadata=md,
                                         plotDict=plotDict, summaryMetrics=summaryMetrics)
    # discoveries via high velocity - more than 1.5 deg/day
    metric = mm.HighVelocityMetric(velocity=highvelocity)
    md = metadata + ' year %d, velocity > %.1f deg/day' %(yr, highvelocity)
    velocitybundles[yr] = mmb.MoMetricBundle(metric, slicer, constraint,
                                         runName=runName, metadata=md,
                                         plotDict=plotDict, summaryMetrics=summaryMetrics)

In [7]:
# In year twelve only, look at discovery as function of SNR
snrbundles = {}
snrList = [4, 5, 5.5, 6, 7]
for snr in snrList:
    metric = mm.DiscoveryChancesMetric(tNight=90./60./24., nObsPerNight=2,
                                      nNightsPerWindow=3, tWindow=15, snrLimit=snr)
    slicer = mos
    constraint = 'night < %d' %(12*365+1)
    md = metadata + ' year %d, SNR=%.1f' %(yr, snr)
    snrbundles[snr] = mmb.MoMetricBundle(metric, slicer, constraint,
                                         runName=runName, metadata=md,
                                         plotDict=plotDict, summaryMetrics=summaryMetrics)

In [ ]:
# Combine all the bundles above into one big dictionary.
bdict = {}
count = 0
for md in [discbundles, extrapairbundles, longdiscbundles, magicbundles, velocitybundles, snrbundles]:
    for mb in md.itervalues():
        bdict[count] = mb
        count += 1
print bdict.keys()
print bdict[24].metadata

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
MPC PHAs year 15, SNR=7.0 night < 4381


In [ ]:
# Run all the metricbundles. This will take a while - we're computing lots of things. 
bg = mmb.MoMetricBundleGroup(bdict, outDir=testDir)
bg.runAll()

In [ ]:
bg.summaryAll()

In [ ]:
ph = plots.PlotHandler(outDir=testDir, savefig=True)
# Will use a mock metricBundle to generate plots. 
# The completeness values (what we typically want to plot here) are a summary value, not an actual metric value.
# So this little bit of fiddling lets us use the plotHandler as we would normally (with a metricBundle), but with 
# the actual values we want to plot. 
yearColors = {5:'r', 10:'g', 12:'b', 15:'y'}
snrColors = {4:'r', 5:'b', 5.5:'y', 6:'g', 7:'k'}

In [ ]:
# Differential completeness, as function of H (for each of nyear above)
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

for yr in nyears:
    completeness = discbundles[yr].summaryValues['Completeness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    plotDict = {'albedo':albedo, 'label':'Year %d: %.0f%s' %(yr, cval*100, '%'), 
                'color':yearColors[yr], 
                'title':'%s Differential Completeness - 3 pairs in 15 nights' %(runName), 
                'ylabel':'Completeness @ H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='3 pairs in 15 nights', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
# Cumulative completeness, as function of H (for each of nyear above)
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

for yr in nyears:
    completeness = discbundles[yr].summaryValues['CumulativeCompleteness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    plotDict = {'albedo':albedo, 'label':'Year %d: %.0f%s' %(yr, cval*100, '%'), 
                'color':yearColors[yr], 
                'title':'%s Cumulative Completeness - 3 pairs in 15 nights' %(runName), 
                'ylabel':'Completeness <= H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CumulativeCompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='3 pairs in 15 nights', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
# Histogram of discovery chances at a given H, in year 12 standard discovery.
b = discbundles[12]
mVals = b.metricValues.swapaxes(0, 1)[Hidx].filled(0)
plt.hist(mVals[np.where(mVals>0)], bins=100, cumulative=True, normed=True, edgecolor='none')
plt.xlabel('# discovery chances')
plt.ylabel('Fraction of NEOs')
plt.xlim(0, 20)
plt.ylim(0, 1)
plt.title('NEOs with X or more discovery chances @ H=%.1f' %(Hmark))
plt.savefig(os.path.join(testDir, '%s_DiscoveryChances22_hist.pdf' %(runName)), format='pdf')

In [ ]:
# Plot the fraction of objects with only X discovery chances, in year 12 of survey in standard discovery.
b = discbundles[12]
Hrange = mos.Hrange
nchances = np.arange(1, 4, 1)
mVals = b.metricValues.swapaxes(0, 1).filled(0)
for n in nchances:
    fraction = np.zeros(len(Hrange), float)
    for i, (H, mVal) in enumerate(zip(Hrange, mVals)):  
        match = np.where(mVal>0)
        n_available = float(len(match[0]))
        fraction[i] = len(np.where(mVal[match]<=n)[0]) / n_available
    plt.plot(Hrange, fraction, label='N chances %d' %(n))
plt.legend(loc='upper left', fancybox=True)
#plt.ylim(0, 5)
plt.xlim(15, Hrange.max())
plt.xlabel('H mag')
plt.ylabel('Fraction of NEOs @H')
plt.title('NEOs with <= N chances of discovery')
plt.savefig(os.path.join(testDir, '%s_FractionNDiscoveryChances.pdf' %(runName)), format='pdf')

In [ ]:
# Cumulative completeness, as function of H (for each of nyear above) -- requiring 4 pairs in 20 nights.
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

for yr in nyears:
    completeness = extrapairbundles[yr].summaryValues['CumulativeCompleteness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    plotDict = {'albedo':albedo, 'label':'Year %d: %.0f%s' %(yr, cval*100, '%'), 
                'color':yearColors[yr], 
                'title':'%s Cumulative Completeness - 4 pairs in 20 nights' %(runName), 
                'ylabel':'Completeness <= H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CumulativeCompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='4 pairs in 20 nights', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
# Cumulative completeness - discovery with 'high' velocity.
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

for yr in nyears:
    completeness = velocitybundles[yr].summaryValues['CumulativeCompleteness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    plotDict = {'albedo':albedo, 'label':'Year %d: %.0f%s' %(yr, cval*100, '%'), 
                'color':yearColors[yr], 
                'title':'%s Fraction of objects seen with velocity>%.1fdeg/day' %(runName, highvelocity), 
                'ylabel':'Completeness <= H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CumulativeCompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='high velocity detections', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
# Fraction found with high velocity but not found with 'discovery'. 
# TBD. Should be able to take metricValues from discbundles and match against metricValues from highVelocity.
combobundles = {}
for yr in nyears:
    found_data = discbundles[yr].metricValues.filled(0) + velocitybundles[yr].metricValues.filled(0)
    found_mask = np.where(discbundles[yr].metricValues.mask & velocitybundles[yr].metricValues.mask,
                          1, 0)
    found_combined = ma.MaskedArray(data=found_data, mask=found_mask, fill_value=0)
    md = metadata + ' Year %d: velocity + MOPS' %(yr)
    combobundles[yr] = mmb.MoMetricBundle(mm.DiscoveryChancesMetric(), slicer=mos, 
                                          constraint=discbundles[yr].constraint, 
                                          runName=runName, metadata=md, summaryMetrics=summaryMetrics)
    combobundles[yr].metricValues = found_combined
    combobundles[yr].computeSummaryStats()

# Plot completeness for 'both'.
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

for yr in nyears:
    completeness = combobundles[yr].summaryValues['CumulativeCompleteness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    plotDict = {'albedo':albedo, 'label':'Year %d: %.0f%s' %(yr, cval*100, '%'), 
                'color':yearColors[yr], 
                'title':'%s Cumulative Completeness - velocity + MOPS' %(runName), 
                'ylabel':'Completeness <= H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CumulativeCompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='velocity+mops', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
# Cumulative completeness, as function of H (for each of nyear above) -- requiring 3 pairs in 30 nights.
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

for yr in nyears:
    completeness = longdiscbundles[yr].summaryValues['CumulativeCompleteness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    plotDict = {'albedo':albedo, 'label':'Year %d: %.0f%s' %(yr, cval*100, '%'), 
                'color':yearColors[yr], 
                'title':'%s Cumulative Completeness - 3 pairs in 30 nights' %(runName), 
                'ylabel':'Completeness <= H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CumulativeCompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='3 pairs in 30 nights', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
# Cumulative completeness, as function of H (for each of nyear above) -- with Mario's magic discovery software.
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

for yr in nyears:
    completeness = magicbundles[yr].summaryValues['CumulativeCompleteness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    plotDict = {'albedo':albedo, 'label':'Year %d: %.0f%s' %(yr, cval*100, '%'), 
                'color':yearColors[yr], 
                'title':'%s Cumulative Completeness - 6 obs in 60 nights' %(runName), 
                'ylabel':'Completeness <= H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CumulativeCompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='6 obs in 60 nights', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
# Cumulative completeness as we vary SNR required for discovery.
compbundles = []

Hmark = 22
Hidx = np.where(mos.Hrange == Hmark)[0]

cvals = np.zeros(len(snrList), float)

for i, snr in enumerate(snrList):
    completeness = snrbundles[snr].summaryValues['CumulativeCompleteness'][0]
    completenessma = ma.MaskedArray(data=completeness, mask=np.zeros(len(completeness)), fill_value=0)
    cval = completeness[Hidx]
    cvals[i] = cval*100
    plotDict = {'albedo':albedo, 'label':'SNR %.1f: %.0f%s' %(snr, cval*100, '%'), 
                'color':snrColors[snr], 
                'title':'%s Cumulative Completeness @ year 12' %(runName), 
                'ylabel':'Completeness <= H', 'yMin':0, 'yMax':1, 'Hmark':22}
    mb = mmb.MoMetricBundle(ms.CumulativeCompletenessMetric(), slicer=mos, constraint=None,
                            runName=runName, metadata='completeness with snr', plotDict=plotDict)
    mb.metricValues = completenessma
    compbundles.append(mb)
ph.setMetricBundles(compbundles)
ph.plot(plotFunc=moPlots.MetricVsH())

In [ ]:
plt.figure()
plt.plot(snrList, cvals)
plt.ylim(70, 90)
plt.xlabel('SNR')
plt.ylabel('Cumulative completeness at H<=22')
plt.title('%s Completeness as function of SNR' %runName)
plt.savefig(os.path.join(testDir, '%s_completenessSNR.pdf' %(runName)), format='pdf')